### Preamble

In [189]:
# Configure libraries

%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
import geohash
from sklearn.metrics import mean_squared_error
import random

In [190]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_
    print "Best score: ", gs.best_score_
    best = gs.best_estimator_
    return best

###  Reading the data

For a Geohash precision of 6 (geohash length of 6 characters), we have roughly 300,000 records. For a precision of 7, we have about 4 million records.

In [191]:
# Each line is of the format:
# ((time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend, geohash), number of pickups)
names = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat", "day_num", "day_cos", "day_sin", "weekend", "geohash", "pickups"]
dftaxi=pd.read_csv("./data/taxi_data_prec_7.csv", header=None, names = names)
print dftaxi.shape

(3819998, 11)


### Define training and test sets

In [192]:
itrain, itest = train_test_split(xrange(dftaxi.shape[0]), train_size=0.8)
mask=np.ones(dftaxi.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([False,  True,  True,  True,  True,  True, False,  True, False,  True], dtype=bool)

### Final preperation for machine learning

I tried initially doing one-hot-encoding on the geohashes, but I quickly realized that this was not feasible from a memory perspective. 3 million records times 100,000 features would not fit in memory. So I decided to go for the numerical latitude and longitude route. Using a random forest, we can easily detect higher order structures in these two variables.

In [193]:
# Split off the features
Xnames = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat",
          "day_num", "day_cos", "day_sin", "weekend", "geohash"]
X = dftaxi[Xnames]

# Split off the target (which will be the logarithm of the number of pickups (+1))
y = np.log10(dftaxi['pickups']+1)

In [194]:
# Get the longitude and latitude from the geohash
def decodegeo(geo, which):
    if len(geo) >= 6:
        geodecoded = geohash.decode(geo)
        return geodecoded[which]
    else:
        return 0
X['latitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 0))
X['longitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 1))
X.head()

,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,geohash,latitude,longitude
0,15:30,0.656250,-0.555570,-0.831470,Monday,0.093750,0.831470,0.555570,0,dr5rvnz,40.775070,-73.949661
1,15:00,0.635417,-0.659346,-0.751840,Tuesday,0.233631,0.102669,0.994716,0,dr5rm12,40.656967,-73.959274
2,08:00,0.343750,-0.555570,0.831470,Wednesday,0.334821,-0.508075,0.861313,0,dr5resv,40.720139,-74.018326
3,14:30,0.614583,-0.751840,-0.659346,Monday,0.087798,0.851662,0.524092,0,dr5russ,40.762711,-73.975754
4,11:30,0.489583,-0.997859,0.065403,Tuesday,0.212798,0.231627,0.972805,0,dr72m1p,40.831375,-73.949661


In [195]:
# Create indicator variables for the hours and days of the week and drop the categorical values
# g = 5
# X = X.join(pd.get_dummies(X['time_cat']))\
#      .join(pd.get_dummies(X['day_cat']))\
#      .drop(['time_cat','day_cat','geohash'], axis=1)
X = X.drop(['time_cat','day_cat','geohash'], axis=1)
#     .join(pd.get_dummies(X['geohash'].str[:g]))\

### Get the training and test sets

In [196]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
print Xtrain.shape
max_samples = 1000000
if Xtrain.shape[0] > max_samples:
    rows = random.sample(Xtrain.index, max_samples)
    Xtrain = Xtrain.ix[rows]
    ytrain = ytrain.ix[rows]
print Xtrain.shape
Xtrain.head()

(3055998, 9)
(1000000, 9)


,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,latitude,longitude
1360274,0.572917,-0.896873,-0.442289,0.224702,0.158281,0.987394,0,40.736618,-73.878250
1363421,0.885417,0.751840,-0.659346,0.983631,0.994716,-0.102669,1,40.806656,-73.956528
3517805,0.843750,0.555570,-0.831470,0.549107,-0.952775,-0.303677,0,40.765457,-74.022446
1417738,0.239583,0.065403,0.997859,0.462798,-0.972805,0.231627,0,40.759964,-73.970261
1396909,0.593750,-0.831470,-0.555570,0.513393,-0.996461,-0.084051,0,40.717392,-73.940048


### Random Forest Regression

In [197]:
# Create a Random Forest Regression estimator
estimator = RandomForestRegressor(n_estimators=20, n_jobs=-1)

The step below takes about 25 mins on my laptop for 300,000 records and up to 100 estimators.

In [208]:
%%time
# Define a grid of parameters over which to optimize the random forest
# We will figure out which number of trees is optimal
parameters = {"n_estimators": [50],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [50]}
best = cv_optimize(estimator, parameters, Xtrain, ytrain, n_folds=5, score_func='mean_squared_error', verbose=3)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.037116 - 2.4min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.037396 - 2.5min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.037323 - 2.7min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.037099 - 2.8min
[CV] max_features=auto, n_estimators=50, max_depth=50 ................
[CV]  max_features=auto, n_estimators=50, max_depth=50, score=-0.037165 - 2.6min

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  2.4min
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 13.1min finished



BEST {'max_features': 'auto', 'n_estimators': 50, 'max_depth': 50} -0.0372196114629 [mean: -0.03722, std: 0.00012, params: {'max_features': 'auto', 'n_estimators': 50, 'max_depth': 50}] make_scorer(mean_squared_error, greater_is_better=False)
Best score:  -0.0372196114629
Wall time: 16min 41s


### Evaluate the results

In [209]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
reg=best.fit(Xtrain, ytrain)
training_accuracy = reg.score(Xtrain, ytrain)
test_accuracy = reg.score(Xtest, ytest)
print "############# based on standard predict ################"
print "R^2 on training data: %0.4f" % (training_accuracy)
print "R^2 on test data:     %0.4f" % (test_accuracy)

############# based on standard predict ################
R^2 on training data: 0.9928
R^2 on test data:     0.9505


Sanity check on some records.

In [210]:
# Show some of the predictions vs. the real number of pickups
# predictions vs. real number of pickups
np.round(np.power(10,np.column_stack((reg.predict(Xtest),ytest))) - 1,decimals=0).astype(int)

array([[189, 178],
       [  1,   1],
       [ 47,  48],
       ..., 
       [  2,   1],
       [ 13,  42],
       [  1,   1]])

In [212]:
# Calculate the Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(reg.predict(Xtest),ytest))
print "RMSE = %0.3f (this is in log-space!)" % rmse
print "So two thirds of the records would be a factor of less than %0.2f away from the real value." % np.power(10,rmse)

RMSE = 0.183 (this is in log-space!)
So two thirds of the records would be a factor of less than 1.52 away from the real value.


In [213]:
# What are the most important features?
import operator
dict_feat_imp = dict(zip(list(X.columns.values),reg.feature_importances_))

sorted_features = sorted(dict_feat_imp.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

[('longitude', 0.49286711184864612),
 ('latitude', 0.42160153852351745),
 ('time_num', 0.02934174261881017),
 ('time_cos', 0.013840678747627112),
 ('day_sin', 0.012299318524748769),
 ('day_num', 0.011275554374888259),
 ('time_sin', 0.010335042662988656),
 ('day_cos', 0.0077883669738441274),
 ('weekend', 0.00065064572492942946)]

### Create predictions for visualization

Now we are going to generate predictions that we can visualize in Tableau. We do this by generating all possible combinations of time and location so that we have a well filled space of predictions. Then we generate predictions for all these combinations and then export to .csv.

#### First we need a dataframe with all possible combinations of time and location

In [228]:
# Construct dataframes with all possible times (time_data) and all possible locations (loc_data)

# Columns about time
time_cols = list(X.columns.values)
time_cols.remove('latitude')
time_cols.remove('longitude')

# Columns about location
loc_cols = ['latitude', 'longitude']

# Unique times
time_data = X.drop(loc_cols, axis=1).drop_duplicates()

# In Tableau we are only going to look at Monday
time_data = time_data[time_data['day_num'] <= 1/7.]

# Unique locations
loc_data = X.drop(time_cols, axis=1).drop_duplicates()

# To reduce memory consumption in Tableau, we are only predicting for
# the region closely around Manhattan and the La Guardia and JFK airports
loc_data = loc_data[(loc_data['latitude'] > 40.5) & (loc_data['latitude'] < 41.1) &
                    (loc_data['longitude'] > -74.1) & (loc_data['longitude'] < -73.6)]

In [224]:
# Dummy column to be able to join them together
time_data['key'] = 1
loc_data['key'] = 1

# Merge the time_data and location_data
result = pd.merge(time_data, loc_data, on='key').drop(['key'], axis=1)
result = result[Xtrain.columns.values]

#### Then we do the prediction

In [225]:
# Get the real number of pickups and take care that we can merge it with the predictions,
# by also taking the geohash and the timestamp
yy = dftaxi[['geohash','day_num','pickups']]

# Decode the geohash in the latitude and longitude
yy['latitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 0))
yy['longitude'] = yy['geohash'].apply(lambda geo: decodegeo(geo, 1))

# Do predictions and convert the logarithm to the normal numbers
result['pred_pickups'] = np.power(10,reg.predict(result)) - 1

# Merge the predictions and the real pickups
result = pd.merge(result, yy, how='left', on=['day_num', 'latitude', 'longitude']).drop(['geohash'], axis=1)
result.head(10)

C:\Users\m07e036.EUROPE.003\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\m07e036.EUROPE.003\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,latitude,longitude,pred_pickups,pickups
0,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.775070,-73.949661,189.259397,178
1,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.656967,-73.959274,3.963223,7
2,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.720139,-74.018326,1.084932,NaN
3,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.762711,-73.975754,677.696195,646
4,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.831375,-73.949661,2.245625,3
5,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.806656,-73.942795,182.849692,168
6,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.707779,-74.021072,1.049252,NaN
7,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.670700,-73.981247,3.682788,1
8,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.735245,-73.875504,42.386296,41
9,0.65625,-0.55557,-0.83147,0.09375,0.83147,0.55557,0,40.720139,-74.001846,486.831538,555


In [226]:
# Drop unnecessary columns to reduce memory consumption in Tableau
result = result.drop(['time_cos','day_num','time_sin','day_cos','day_sin','weekend'], axis=1)

In [227]:
# Write to csv
result.to_csv('./data/taxi-data-predictions_prec_7_monday.csv')